# Práctica Bases de Datos

## Parte I: SQL Scape Room 
Por parejas, resuelve el misterio del asesinato en http://mystery.knightlab.com/

## Parte II: Modelo Relacional

La empresa de venta de coches **KARLS** quiere crear un modelo de bases de datos. Para ello, nos ha compartido el fichero `ventas.csv`, que contiene sus ventas más recientes, con los siguientes campos:

*	no_orden: número de la orden
*	codigo_producto: código del producto comprado
*	cantidad_orden: cantidad comprada
*	precio_cada: precio de venta por unidad 
*	nombre_producto: nombre del producto
*	linea_productos: línea de productos
*	descr: descripción de la línea de productos
*	precio_compra: precio de compra de cada producto al proveedor
*	fecha_orden: fecha de la compra
*	no_cliente: número del cliente
*	nombre_cliente: nombre del cliente
*	tlf: teléfono del cliente
*	direccion: dirección del cliente
*	ciudad: ciudad del cliente
*	estado: estado o provincia del cliente
*	codigo_postal: código postal del cliente
*	pais: país del cliente

Por otro lado, KARLS quiere conocer el idioma de cada cliente para poder realizar campañas de marketing personalizadas en diversos idiomas. Para conocer el idioma oficial de cada país, disponemos del acceso a una base de datos con los siguientes parámetros:

*	Servidor: relational.fit.cvut.cz
*	Usuario: guest
*	Contraseña: relational
*	Nombre de la base de datos: world

![imagen](https://relational.fit.cvut.cz/assets/img/datasets-generated/world.svg)


In [2]:
import sqlite3
import pymysql
import pandas as pd

In [3]:
# Mostramos los datos del .csv en pandas
datos = pd.read_csv('ventas.csv')
datos

,no_orden,codigo_producto,cantidad_orden,precio_cada,nombre_producto,linea_productos,descr,precio_compra,fecha_orden,no_cliente,nombre_cliente,tlf,direccion,ciudad,estado,codigo_postal,pais
0,10100,S18_1749,30,136.00,1917 Grand Touring Sedan,Vintage Cars,Our Vintage Car models realistically portray a...,86.70,2003-01-06,363,Online Diecast Creations Co.,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,10100,S18_2248,50,55.09,1911 Ford Town Car,Vintage Cars,Our Vintage Car models realistically portray a...,33.30,2003-01-06,363,Online Diecast Creations Co.,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
2,10100,S18_4409,22,75.46,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,Our Vintage Car models realistically portray a...,43.26,2003-01-06,363,Online Diecast Creations Co.,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
3,10100,S24_3969,49,35.29,1936 Mercedes Benz 500k Roadster,Vintage Cars,Our Vintage Car models realistically portray a...,21.75,2003-01-06,363,Online Diecast Creations Co.,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
4,10101,S18_2325,25,108.06,1932 Model A Ford J-Coupe,Vintage Cars,Our Vintage Car models realistically portray a...,58.48,2003-01-09,128,"Blauer See Auto, Co.",+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,NaN,60528,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,1962 Volkswagen Microbus,Trucks and Buses,The Truck and Bus models are realistic replica...,61.34,2005-05-31,119,La Rochelle Gifts,40.67.8555,"67, rue des Cinquante Otages",Nantes,NaN,44000,France
2992,10425,S24_2840,31,31.82,1958 Chevy Corvette Limited Edition,Classic Cars,Attention car enthusiasts: Make your wildest c...,15.91,2005-05-31,119,La Rochelle Gifts,40.67.8555,"67, rue des Cinquante Otages",Nantes,NaN,44000,France
2993,10425,S32_1268,41,83.79,1980’s GM Manhattan Express,Trucks and Buses,The Truck and Bus models are realistic replica...,53.93,2005-05-31,119,La Rochelle Gifts,40.67.8555,"67, rue des Cinquante Otages",Nantes,NaN,44000,France
2994,10425,S32_2509,11,50.32,1954 Greyhound Scenicruiser,Trucks and Buses,The Truck and Bus models are realistic replica...,25.98,2005-05-31,119,La Rochelle Gifts,40.67.8555,"67, rue des Cinquante Otages",Nantes,NaN,44000,France


Hay que pensar en las tablas de DIMENSIÓN y HECHOS

In [5]:
# Dimensión CLIENTE
cols = ['no_cliente', 'nombre_cliente', 'tlf', 'direccion', 'ciudad', 'estado', 'codigo_postal','pais']
cliente = datos[cols].drop_duplicates()
cliente

,no_cliente,nombre_cliente,tlf,direccion,ciudad,estado,codigo_postal,pais
0,363,Online Diecast Creations Co.,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
4,128,"Blauer See Auto, Co.",+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,NaN,60528,Germany
8,181,Vitachrome Inc.,2125551500,2678 Kingston Rd.,NYC,NY,10022,USA
10,121,Baane Mini Imports,07-98 9555,Erling Skakkes gate 78,Stavern,NaN,4110,Norway
26,141,Euro+ Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",Madrid,NaN,28034,Spain
...,...,...,...,...,...,...,...,...
1619,204,Online Mini Collectables,6175557555,7635 Spinnaker Dr.,Brickhaven,MA,58339,USA
1650,249,Amica Models & Co.,011-4988555,Via Monte Bianco 34,Torino,NaN,10100,Italy
1708,299,"NorwayGifts By Mail, Co.",+47 2212 1555,Drammensveien 126A,Oslo,NaN,N 0106,Norway
1721,286,Marta's Replicas Co.,6175558555,39323 Spinnaker Dr.,Cambridge,MA,51247,USA


In [6]:
# Dimensión PRODUCTOS
cols = ['codigo_producto', 'nombre_producto','linea_productos','descr','precio_compra']
producto = datos[cols].drop_duplicates()
producto

,codigo_producto,nombre_producto,linea_productos,descr,precio_compra
0,S18_1749,1917 Grand Touring Sedan,Vintage Cars,Our Vintage Car models realistically portray a...,86.70
1,S18_2248,1911 Ford Town Car,Vintage Cars,Our Vintage Car models realistically portray a...,33.30
2,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,Our Vintage Car models realistically portray a...,43.26
3,S24_3969,1936 Mercedes Benz 500k Roadster,Vintage Cars,Our Vintage Car models realistically portray a...,21.75
4,S18_2325,1932 Model A Ford J-Coupe,Vintage Cars,Our Vintage Car models realistically portray a...,58.48
...,...,...,...,...,...
110,S24_1628,1966 Shelby Cobra 427 S/C,Classic Cars,Attention car enthusiasts: Make your wildest c...,29.18
113,S24_2766,1949 Jaguar XK 120,Classic Cars,Attention car enthusiasts: Make your wildest c...,47.25
114,S24_2887,1952 Citroen-15CV,Classic Cars,Attention car enthusiasts: Make your wildest c...,72.82
115,S24_3191,1969 Chevrolet Camaro Z28,Classic Cars,Attention car enthusiasts: Make your wildest c...,50.51


In [8]:
# TABLA DE HECHOS: COMPRAS
cols = ['no_orden', 'codigo_producto', 'cantidad_orden', 'precio_cada', 'fecha_orden','no_cliente']
compras = datos[cols].drop_duplicates()
compras

,no_orden,codigo_producto,cantidad_orden,precio_cada,fecha_orden,no_cliente
0,10100,S18_1749,30,136.00,2003-01-06,363
1,10100,S18_2248,50,55.09,2003-01-06,363
2,10100,S18_4409,22,75.46,2003-01-06,363
3,10100,S24_3969,49,35.29,2003-01-06,363
4,10101,S18_2325,25,108.06,2003-01-09,128
...,...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,2005-05-31,119
2992,10425,S24_2840,31,31.82,2005-05-31,119
2993,10425,S32_1268,41,83.79,2005-05-31,119
2994,10425,S32_2509,11,50.32,2005-05-31,119


Creamos una base de datos inicialmente vacía

In [9]:
conn = sqlite3.connect('karls.sqlite')

Guardamos nuestras tablas en la base de datos

In [14]:
cliente.to_sql('cliente', conn, index=False, if_exists='replace')

In [28]:
pd.read_sql('SELECT * FROM cliente', conn)

,no_cliente,nombre_cliente,tlf,direccion,ciudad,estado,codigo_postal,pais
0,363,Online Diecast Creations Co.,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,128,"Blauer See Auto, Co.",+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany
2,181,Vitachrome Inc.,2125551500,2678 Kingston Rd.,NYC,NY,10022,USA
3,121,Baane Mini Imports,07-98 9555,Erling Skakkes gate 78,Stavern,None,4110,Norway
4,141,Euro+ Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",Madrid,None,28034,Spain
...,...,...,...,...,...,...,...,...
93,204,Online Mini Collectables,6175557555,7635 Spinnaker Dr.,Brickhaven,MA,58339,USA
94,249,Amica Models & Co.,011-4988555,Via Monte Bianco 34,Torino,None,10100,Italy
95,299,"NorwayGifts By Mail, Co.",+47 2212 1555,Drammensveien 126A,Oslo,None,N 0106,Norway
96,286,Marta's Replicas Co.,6175558555,39323 Spinnaker Dr.,Cambridge,MA,51247,USA


In [16]:
producto.to_sql('producto', conn, index=False, if_exists='replace')

In [17]:
compras.to_sql('compras', conn, index=False, if_exists='replace')

Nos conectamos a la base de datos world y añadimos la tabla que relacione cada cliente con su idioma

In [18]:
database_host = 'relational.fit.cvut.cz'
username = 'guest'
password = 'relational'
database_name = 'world'

db = pymysql.connect(database_host,username,password,database_name)

In [19]:
pd.read_sql('SHOW TABLES',db)

,Tables_in_world
0,City
1,Country
2,CountryLanguage


In [20]:
pd.read_sql('SELECT * FROM CountryLanguage', db)

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9
...,...,...,...,...
979,ZMB,Tongan,F,11.0
980,ZWE,English,T,2.2
981,ZWE,Ndebele,F,16.2
982,ZWE,Nyanja,F,2.2


In [50]:
# Query para saber el idioma más hablado de cada país 

query = '''
SELECT Name, Language, IsOfficial, Percentage

FROM CountryLanguage
JOIN Country
ON CountryLanguage.CountryCode = Country.Code

WHERE Percentage = (SELECT MAX(Percentage) 
                        FROM CountryLanguage
                        WHERE Country.Code = CountryLanguage.CountryCode
                    )
'''

idiomas = pd.read_sql(query, db)
idiomas

,Name,Language,IsOfficial,Percentage
0,Aruba,Papiamento,F,76.7
1,Afghanistan,Pashto,T,52.4
2,Angola,Ovimbundu,F,37.2
3,Anguilla,English,T,0.0
4,Albania,Albaniana,T,97.9
...,...,...,...,...
236,Yemen,Arabic,T,99.6
237,Yugoslavia,Serbo-Croatian,T,75.2
238,South Africa,Zulu,T,22.7
239,Zambia,Bemba,F,29.7


In [51]:
idiomas.to_sql('idiomas', conn, index=False, if_exists='replace')

In [52]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',conn)

,name
0,cliente
1,producto
2,compras
3,idiomas_cliente
4,idiomas


In [53]:
# Relacionar a cada cliente con su idioma
query = '''
SELECT no_cliente, pais, Language

FROM cliente
LEFT JOIN idiomas
ON cliente.pais = idiomas.Name
WHERE Language IS null
'''

pd.read_sql(query, conn)

,no_cliente,pais,Language
0,363,USA,None
1,181,USA,None
2,131,USA,None
3,486,USA,None
4,187,UK,None
5,129,USA,None
6,124,USA,None
7,424,USA,None
8,112,USA,None
9,151,USA,None


In [54]:
# Observamos que hay clientes (de USA y UK) que no tienen idioma asociado. Comprobamos que se debe a que en la base de datos
# world, los nombres de estos países no coinciden

query= '''
SELECT * 
FROM idiomas
WHERE Name LIKE '%United%'
'''

pd.read_sql(query, conn)

,Name,Language,IsOfficial,Percentage
0,United Arab Emirates,Arabic,T,42.0
1,United Kingdom,English,T,97.3
2,United States Minor Outlying Islands,English,T,0.0
3,United States,English,T,86.2


In [55]:
# Modifico la tabla idiomas
query = '''
SELECT  CASE WHEN Name="United States" THEN "USA"
             WHEN Name = "United Kingdom" THEN "UK"
             ELSE Name END AS Name,
        Language,
        IsOfficial,
        Percentage

FROM CountryLanguage
JOIN Country
ON CountryLanguage.CountryCode = Country.Code

WHERE Percentage = (SELECT MAX(Percentage) 
                        FROM CountryLanguage
                        WHERE Country.Code = CountryLanguage.CountryCode
                    )
'''

idiomas = pd.read_sql(query, db)
idiomas

,Name,Language,IsOfficial,Percentage
0,Aruba,Papiamento,F,76.7
1,Afghanistan,Pashto,T,52.4
2,Angola,Ovimbundu,F,37.2
3,Anguilla,English,T,0.0
4,Albania,Albaniana,T,97.9
...,...,...,...,...
236,Yemen,Arabic,T,99.6
237,Yugoslavia,Serbo-Croatian,T,75.2
238,South Africa,Zulu,T,22.7
239,Zambia,Bemba,F,29.7


In [56]:
idiomas.to_sql('idiomas', conn, index=False, if_exists='replace')

In [57]:
# Relacionar a cada cliente con su idioma
query = '''
SELECT no_cliente, pais, Language

FROM cliente
LEFT JOIN idiomas
ON cliente.pais = idiomas.Name
'''

idiomas_cliente = pd.read_sql(query, conn)

In [58]:
pd.set_option('display.max_rows',100)
idiomas_cliente

,no_cliente,pais,Language
0,363,USA,English
1,128,Germany,German
2,181,USA,English
3,121,Norway,Norwegian
4,141,Spain,Spanish
5,145,Denmark,Danish
6,278,Italy,Italian
7,131,USA,English
8,385,Philippines,Pilipino
9,486,USA,English


In [59]:
idiomas_cliente.to_sql('idiomas_cliente',conn, index=False, if_exists='replace')

In [60]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',conn)

,name
0,cliente
1,producto
2,compras
3,idiomas
4,idiomas_cliente


 Puedes generar el diagrama de la base de datos en https://dbdiagram.io/d
 <img src="https://i.ibb.co/4PXndfB/Ejercicio2.png">

¿Cuál es el producto que aporta más beneficios?

In [61]:
query ='''
SELECT nombre_producto, SUM(cantidad_orden*(precio_cada-precio_compra)) AS Beneficio

FROM compras
JOIN producto
ON compras.codigo_producto = producto.codigo_producto

GROUP BY nombre_producto
ORDER BY Beneficio DESC
'''

pd.read_sql(query, conn)

,nombre_producto,Beneficio
0,1992 Ferrari 360 Spider red,135996.78
1,1952 Alpine Renault 1300,95282.58
2,2001 Ferrari Enzo,93349.65
3,2003 Harley-Davidson Eagle Drag Bike,81031.30
4,1968 Ford Mustang,72579.26
...,...,...
104,1930 Buick Marquette Phaeton,12536.80
105,1936 Mercedes Benz 500k Roadster,11841.39
106,1982 Ducati 996 R,11397.92
107,Boeing X-32A JSF,11233.33
